# Common Voice Dataset Training with Whisper

This notebook demonstrates how to train or fine-tune the Whisper model using the Common Voice dataset.

In [2]:
%pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached starlette-0.46.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   --------- ------------------------------ 2.4/10.4 MB 12.2 MB/s eta 0:00:01
   ------------------ --------------------- 4.7/10.4 MB 11.9 MB/s eta 0:00:01
   --------------------------- ------------ 7.1/10.4 MB 11.8 MB/s eta 0:00:01
   ------------------------------------ --- 9.4/10.4 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 10.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ----------------- ---------------------- 2.4/5.5 MB 12.2 MB/s 

In [4]:
import os
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from src.config.hyperparameters import Hyperparameters
from src.models.whisper.model import WhisperModel
from src.utils.common_voice import process_common_voice_metadata
from datasets import load_dataset, DatasetDict

# Log into HuggingFace

In [11]:
from huggingface_hub import notebook_login

notebook_login()

# Import datasets

In [14]:
from datasets import load_dataset, DatasetDict, DownloadConfig

common_voice = DatasetDict()
config = DownloadConfig(resume_download=True)

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "es", split="train", trust_remote_code=True, download_config=config)
common_voice["validation"] = load_dataset("mozilla-foundation/common_voice_11_0", "es", split="validation", trust_remote_code=True, download_config=config)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "es", split="test", trust_remote_code=True, download_config=config)

print(common_voice)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)bf6034f75ee0514e1bf46b923a14dc798b74c0b3:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)714862aeda48d7ff99e4a480c9ac2f4e32219a8a:   0%|          | 0.00/984M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Generating other split: 0 examples [00:00, ? examples/s]

Reading metadata...: 1180383it [00:08, 145942.18it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]

Reading metadata...: 52095it [00:00, 126766.31it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 230467
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 15520
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 15520
    })
})


In [15]:
from datasets import concatenate_datasets
common_voice['train_full'] = concatenate_datasets([common_voice['train'], common_voice['validation']])

# Preprocess data

In [17]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Spanish", task="transcribe")
print(common_voice["train_full"][0])

{'client_id': '34719bb7c7344da7733b85c9d7215d24326093f1a2cd3a445bdc6dfe9ec4a8c9fe9729a73f6c29764545276bff81ffa65d3944f6da7a3ee3c06d0eb124fac797', 'path': 'C:\\Users\\PC\\.cache\\huggingface\\datasets\\downloads\\extracted\\d23b4c86f4c9ac20e0a765eb29f593dcfe5f2b57e5776ffde9ee387f4e75c807\\es_train_0/common_voice_es_18338585.mp3', 'audio': {'path': 'C:\\Users\\PC\\.cache\\huggingface\\datasets\\downloads\\extracted\\d23b4c86f4c9ac20e0a765eb29f593dcfe5f2b57e5776ffde9ee387f4e75c807\\es_train_0/common_voice_es_18338585.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -3.25855126e-06, -3.52389725e-06, -3.05285812e-06]), 'sampling_rate': 48000}, 'sentence': '¿ Qué tal a tres de cinco ?', 'up_votes': 2, 'down_votes': 1, 'age': '', 'gender': '', 'accent': '', 'locale': 'es', 'segment': ''}


In [18]:
from datasets import Audio
common_voice = common_voice.remove_columns(['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'path', 'segment', 'up_votes'])
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
print(common_voice["train_full"][0])

{'audio': {'path': 'C:\\Users\\PC\\.cache\\huggingface\\datasets\\downloads\\extracted\\d23b4c86f4c9ac20e0a765eb29f593dcfe5f2b57e5776ffde9ee387f4e75c807\\es_train_0/common_voice_es_18338585.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        7.67193796e-07, -4.82132691e-07, -3.25116753e-06]), 'sampling_rate': 16000}, 'sentence': '¿ Qué tal a tres de cinco ?'}


In [ ]:
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small", language="Spanish", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Spanish", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Spanish", task="transcribe")

def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [20]:
common_voice = common_voice.map(
    prepare_dataset,
    remove_columns=common_voice.column_names["train_full"],
    desc="Processing audio files",
)

Processing audio files:   0%|          | 0/230467 [00:00<?, ? examples/s]

Processing audio files:   0%|          | 0/15520 [00:00<?, ? examples/s]

Processing audio files:   0%|          | 0/15520 [00:00<?, ? examples/s]

Processing audio files:   0%|          | 0/245987 [00:00<?, ? examples/s]

# Training

In [22]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.generation_config.language = "es"
model.generation_config.task = "transcribe"
model.generation_config.force_decoder_ids = None

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [23]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [24]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [25]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


c:\Users\PC\.conda\envs\voxlens_stt\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\PC\.conda\envs\voxlens_stt\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    # evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    # load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


In [32]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

C:\Users\PC\AppData\Local\Temp\ipykernel_24704\240849381.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [33]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss
25,0.872300
50,0.825800
75,0.598700
100,0.427200
125,0.314400
150,0.284200
175,0.286200
200,0.276100
225,0.293600
250,0.289100


KeyboardInterrupt: 

# Upload to HuggingFace so it can be reused

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0", 
    "dataset_args": "",
    "language": "es",
    "model_name": "VoxLens - OpenAI Whisper Small Spanish",
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

trainer.push_to_hub(**kwargs)

# Test the model

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("brauliodev/voxlens")
processor = WhisperProcessor.from_pretrained("brauliodev/voxlens")

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="brauliodev/voxlens")

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small Spanish",
    description="OpenAI Whisper Small Spanish model fine-tuned on Common Voice 11.0",
)

iface.launch()
